In [0]:
!pip3 -qq install torch==0.4.1
!pip -qq install torchtext==0.3.1
!pip -qq install torchvision==0.2.1
!pip -qq install spacy==2.0.16
!python -m spacy download en
!pip install sacremoses==0.0.5
!pip install subword_nmt==0.3.5
!wget -qq http://www.manythings.org/anki/rus-eng.zip 
!unzip rus-eng.zip

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Machine Translation

Мы уже несколько раз смотрели на эту картинку:

![RNN types](http://karpathy.github.io/assets/rnn/diags.jpeg =x250)

*From [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*

В POS tagging мы (ну, некоторые точно) использовали важную идею: сначала некоторой функцией над символами строился эмбеддинг для слова (например, many to one rnn'кой на картинке). Потом другая rnn'ка строила эмбеддинги слов с учетом их контекста. А дальше это всё классифицируется логистической регрессией.

Тут важно то, что мы обучаем encoder для построения эмбеддингов end2end - прямо в составе сети (это основное отличие нейросетей от классических подходов - в умении делать end2end).

Другое, что мы делали - это языковые модели. Вот, типа такого:
![](https://hsto.org/web/dc1/7c2/c4e/dc17c2c4e9ac434eb5346ada2c412c9a.png =x200)

Обратите внимание на красную стрелку - она показывает передачу скрытого состояния, которое отвечает за память сети.

А теперь совместим эти две идеи:

![](https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/g3doc/img/seq2seq.jpg =x400)  
*From [tensorflow/nmt](https://github.com/tensorflow/nmt)*

Выглядит все почти как языковая модель, но в синей части предсказания не делаются, используется только последнее скрытое состояние.

Синяя часть сети называется энкодер, она строит эмбеддинг последовательности. Красная часть - декодер, работает как обычная языковая модель, но учитывает результат работы энкодера.

В итоге, энкодер учится эффективно извлекать смысл из последовательности слов, а декодер должен строить по ним новую последовательность. Это может быть последовательностью слов перевода, или последовательностью слов в ответе чат бота, или еще чем-то в зависимости от вашей испорченности.

## Подготовка данных

Начнем с чтения данных. Возьмем их у anki, поэтому они немного специфичны:

In [0]:
!shuf -n 10 rus.txt

Токенизируем их:

In [0]:
from torchtext.data import Field, Example, Dataset, BucketIterator

BOS_TOKEN = '<s>'
EOS_TOKEN = '</s>'

source_field = Field(tokenize='spacy', init_token=None, eos_token=EOS_TOKEN)
target_field = Field(tokenize='moses', init_token=BOS_TOKEN, eos_token=EOS_TOKEN)
fields = [('source', source_field), ('target', target_field)]

In [0]:
source_field.preprocess("It's surprising that you haven't heard anything about her wedding.")

In [0]:
target_field.preprocess('Удивительно, что ты ничего не слышал о её свадьбе.')

In [0]:
from tqdm import tqdm

MAX_TOKENS_COUNT = 16
SUBSET_SIZE = .3

examples = []
with open('rus.txt') as f:
    for line in tqdm(f, total=328190):
        source_text, target_text = line.split('\t')
        source_text = source_field.preprocess(source_text)
        target_text = target_field.preprocess(target_text)
        if len(source_text) <= MAX_TOKENS_COUNT and len(target_text) <= MAX_TOKENS_COUNT:
            if np.random.rand() < SUBSET_SIZE:
                examples.append(Example.fromlist([source_text, target_text], fields))

Построим датасеты:

In [0]:
dataset = Dataset(examples, fields)

train_dataset, test_dataset = dataset.split(split_ratio=0.85)

print('Train size =', len(train_dataset))
print('Test size =', len(test_dataset))

source_field.build_vocab(train_dataset, min_freq=3)
print('Source vocab size =', len(source_field.vocab))

target_field.build_vocab(train_dataset, min_freq=3)
print('Target vocab size =', len(target_field.vocab))

train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), batch_sizes=(32, 256), shuffle=True, device=DEVICE, sort=False
)

In [0]:
source_field.process([source_field.preprocess("It's surprising that you haven't heard anything about her wedding.")])

In [0]:
source_field.vocab.itos

In [0]:
target_field.vocab.itos

## Seq2seq модель

Пора писать простой seq2seq. Разобьем модель на несколько модулей - Encoder, Decoder и их объединение. 

Encoder должен быть подобен символьной сеточке в POS tagging'е: эмбеддить токены и запускать rnn'ку (в данном случае будем пользоваться GRU) и отдавать последнее скрытое состояние.

Decoder почти такой же, только еще и предсказывает токены на каждом своем шаге.

**Задание** Реализовать модели.

In [0]:
batch = next(iter(train_iter))

In [0]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1, bidirectional=False):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=rnn_hidden_dim, 
                           num_layers=num_layers, bidirectional=bidirectional)

    def forward(self, inputs, hidden=None):
        <implement me>

In [0]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=rnn_hidden_dim, num_layers=num_layers)
        self._out = nn.Linear(rnn_hidden_dim, vocab_size)

    def forward(self, inputs, encoder_output, hidden=None):
        <implement me>

Модель перевода будет просто сперва вызывать Encoder, а потом передавать его скрытое состояние декодеру в качестве начального.

In [0]:
class TranslationModel(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, emb_dim=128, 
                 rnn_hidden_dim=256, num_layers=1, bidirectional_encoder=False):
        
        super().__init__()
        
        self.encoder = Encoder(source_vocab_size, emb_dim, rnn_hidden_dim, num_layers, bidirectional_encoder)
        self.decoder = Decoder(target_vocab_size, emb_dim, rnn_hidden_dim, num_layers)
        
    def forward(self, source_inputs, target_inputs):
        encoder_hidden = self.encoder(source_inputs)
        
        return self.decoder(target_inputs, encoder_hidden, encoder_hidden)

In [0]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), target_vocab_size=len(target_field.vocab)).to(DEVICE)

model(batch.source, batch.target)

Реализуем простой перевод - жадный. На каждом шаге будем выдавать самый вероятный из предсказываемых токенов:

![](https://github.com/tensorflow/nmt/raw/master/nmt/g3doc/img/greedy_dec.jpg =x400)  
*From [tensorflow/nmt](https://github.com/tensorflow/nmt)*

**Задание** Реализовать функцию.

In [0]:
def greedy_decode(model, source_text, source_field, target_field):
    bos_index = target_field.vocab.stoi[BOS_TOKEN]
    eos_index = target_field.vocab.stoi[EOS_TOKEN]
    
    model.eval()
    with torch.no_grad():
        result = [] # list of predicted tokens indices
        <implement me>
            
        return ' '.join(target_field.vocab.itos[ind.squeeze().item()] for ind in result)

In [0]:
greedy_decode(model, "Do you believe?", source_field, target_field)

Нужно как-то оценивать модель.

Обычно для этого используется [BLEU скор](https://en.wikipedia.org/wiki/BLEU) - что-то вроде точности угадывания n-gram из правильного (референсного) перевода.

**Задание** Реализовать функцию оценки: для батчей из `iterator` предсказать их переводы, обрезать по `</s>` и сложить правильные варианты и предсказанные в `refs` и `hyps` соответственно.

In [0]:
from nltk.translate.bleu_score import corpus_bleu

def evaluate_model(model, iterator):
    model.eval()
    refs, hyps = [], []
    eos_index = iterator.dataset.fields['target'].vocab.stoi[EOS_TOKEN]
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            <implement me>
            
    return corpus_bleu([[ref] for ref in refs], hyps) * 100

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.source, batch.target)
                
                target = torch.cat((batch.target[1:], batch.target.new_ones((1, batch.target.shape[1]))))
                loss = criterion(logits.view(-1, logits.shape[-1]), target.view(-1))

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, None, name_prefix + '  Val:')
            print('\nVal BLEU = {:.2f}'.format(evaluate_model(model, val_iter)))

In [0]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), target_vocab_size=len(target_field.vocab)).to(DEVICE)

pad_idx = target_field.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)

optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=30, val_iter=test_iter)

## Scheduled Sampling

До сих пор мы тренировали перевод, используя так называемый *teacher forcing*: в качестве выхода на предыдущем шаге декодер принимал всегда правильный токен. Проблема такого подхода - во время инференса правильный токен, скорее всего, не выберется хотя бы на каком-то шаге. Получится, что сеть училась на хороших входах, использоваться будет на плохом - это легко может всё поломать.

Альтернативный подход - прямо во время обучения сэмплировать токен с текущего шага и передавать его на следующий.

Такой подход не слишком-то обоснован математически (градиенты не прокидываются через сэмплирование), но его интересно реализовать и он зачастую улучшает качество.

**Задание** Обновите `Decoder`: замените вызов rnn'ки над последовательностью на цикл. На каждом шаге вероятностью `p` передавайте в качестве предыдущего выхода декодеру правильный вход, а иначе - argmax от предыдущего выхода (цикл должен быть похожим на те, которые есть в `greedy_decode` и `evaluate_model`). При передаче argmax вызывайте `detach`, чтобы градиенты не прокидывались. Все выходы собирайте в список, в конце сделайте `torch.cat`. 

В результате при вероятности, равной `p=1`, должно получиться как раньше, только медленнее. При обучении можно передавать `p=0.5`, на инференсе - `p=1`.

## Beam Search

Другой способ бороться с ошибками в декодировании на инференсе - делать beam search. По сути это поиск в глубину с очень сильными отсечениями на каждом шаге:

![](https://image.ibb.co/dBRKkA/2018-11-06-13-53-40.png =x300)   
*From [cs224n, Machine Translation, Seq2Seq and Attention](http://web.stanford.edu/class/cs224n/lectures/lecture10.pdf)*

На картинке на каждом шаге выбирается два лучших (согласно предсказаниям сети) из четырех вариантов продолжений цепочек.

Для сравнения бимов используются суммы log-вероятностей токенов, входящих в бим. Чтобы получить log-вероятности, нужно просто вызвать `F.log_softmax` у логитов. Преимущество сложения логарифмов перед умножением вероятностей должно быть понятно: нет таких проблем с численной неустойчивостью - умножая вероятности, близкие к нулю, мы очень шустро получим ноль в качестве скора бима.

В итоге нужно реализовать аналог `gready_decoding`. 

Beam будет состоять из последовательности индексов токенов (в начале - `[bos_index]`),  суммарного качества (в начале 0) и последнего `hidden` (в начале `encoder_hidden`).


Интерактивная визуализация, утащенная у https://github.com/yandexdataschool/nlp_course:

In [0]:
!wget https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/beam_search.html 2> log
from IPython.display import HTML
# source: parlament does not support the amendment freeing tymoshenko
HTML('./beam_search.html')

In [0]:
def beam_search_decode(model, source_text, source_field, target_field, beam_size=5):
    bos_index = target_field.vocab.stoi[BOS_TOKEN]
    eos_index = target_field.vocab.stoi[EOS_TOKEN]
    
    model.eval()
    with torch.no_grad():
        encoder_hidden = model.encoder(...source_text...)
        beams = [([bos_index], 0, encoder_hidden)]
        
        # 1. make next step from each beam
        # 2. create new beams from top beam_size of each continuation (best next token variants for the given token)
        # 3. leave only top beam_size beams
        # 4. repeat
            
        return ' '.join(target_field.vocab.itos[ind.squeeze().item()] for ind in result)

## Улучшения модели

**Задание** Попробуйте повысить качество модели. Попробуйте: 
- Bidirectional encoder
- Dropout
- Stack moar layers

## Byte-Pair Encoding

Мы можем представлять слова одним индексом - и использовать словные эмбеддинги как строки матрицы эмбеддингов.  
Можем считать их набором символов и получать словный эмбеддинг с помощью некоторой функции над символьными эмбеддингами.

Наконец, можем ещё использовать промежуточное представление - как набор подслов.

Несколько лет назад использование подслов предложили для задачи машинного перевода: [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/abs/1508.07909). Там использовалось byte-pair encoding.

По сути это процесс объединения самых частотных пар символов алфавита в новый суперсимвол. Пусть у нас есть словарь, состоящий из такого набора слов:  
`‘low·’, ‘lowest·’, ‘newer·’, ‘wider·’`   
(`·` означает конец слова)

Тогда первым может выучиться новый символ `r·`, за ним `l o` превратится в `lo`. К этому новому символу приклеится `w`: `lo w` $\to$ `low`. И так далее.

Утверждается, что таким образом выучатся, во-первых, все частотные и короткие слова, а во-вторых, все значимые подслова. Например, в полученном в результате алфавите должны найтись `ly·` и `tion·`.

Дальше слово можно разбить на набор подслов - и действовать, как с символами.

Здесь можно найти уже предобученные эмбеддинги: [BPEmb](https://github.com/bheinzerling/bpemb).

Обучим модель для них:

In [0]:
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE

with open('data.en', 'w') as f_src, open('data.ru', 'w') as f_dst:
    for example in examples:
        f_src.write(' '.join(example.source) + '\n')
        f_dst.write(' '.join(example.target) + '\n')

bpe = {}
for lang in ['en', 'ru']:
    with open('./data.' + lang) as f_data, open('bpe_rules.' + lang, 'w') as f_rules:
        learn_bpe(f_data, f_rules, num_symbols=3000)
    with open('bpe_rules.' + lang) as f_rules:
        bpe[lang] = BPE(f_rules)

In [0]:
bpe['en'].process_line(' '.join(examples[10000].source))

In [0]:
bpe['ru'].process_line(' '.join(examples[10000].target))

**Задание** Переобучиться с subword'ами вместо слов. Возможно, поменять их число (`num_symbols`)

# Image Captioning

Не обязательно энкодить последовательности слов. Наример, можно использовать сверточную сеть для энкодера картинки - и генерировать подпись к ней:

![](https://image.ibb.co/fpYdkL/image-captioning.png)  
*From [Image Captioning Tutorial](https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/03-advanced/image_captioning)*

В результате получаются очень прикольные подписи: [https://cs.stanford.edu/people/karpathy/deepimagesent/](https://cs.stanford.edu/people/karpathy/deepimagesent/).

Скачаем данные для обучения:

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '13BP-6Xd6ymhGallRppYfJBO6UUjFCtbB'})
downloaded.GetContentFile('image_codes.npy')

downloaded = drive.CreateFile({'id': '1O7_3lyTyBMXsBBIt1PwUXwLdkyRQzZML'})
downloaded.GetContentFile('sources.txt')

downloaded = drive.CreateFile({'id': '1t-Dy8TzoRuTMoM7N9NJZKgWXfaw3b6KF'})
downloaded.GetContentFile('texts.txt')

!wget http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_Dataset.zip
!unzip Flickr8k_Dataset.zip

Скачем предобученную модельку:

In [0]:
from torchvision.models.inception import Inception3

class BeheadedInception3(Inception3):
    """ Like torchvision.models.inception.Inception3 but the head goes separately """
    
    def forward(self, x):
        x = x.clone()
        x[:, 0] = x[:, 0] * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
        x[:, 1] = x[:, 1] * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
        x[:, 2] = x[:, 2] * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
        x = self.Conv2d_1a_3x3(x)
        x = self.Conv2d_2a_3x3(x)
        x = self.Conv2d_2b_3x3(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = self.Conv2d_3b_1x1(x)
        x = self.Conv2d_4a_3x3(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = self.Mixed_5b(x)
        x = self.Mixed_5c(x)
        x = self.Mixed_5d(x)
        x = self.Mixed_6a(x)
        x = self.Mixed_6b(x)
        x = self.Mixed_6c(x)
        x = self.Mixed_6d(x)
        x = self.Mixed_6e(x)
        x = self.Mixed_7a(x)
        x = self.Mixed_7b(x)
        x_for_attn = x = self.Mixed_7c(x)
        # 8 x 8 x 2048
        x = F.avg_pool2d(x, kernel_size=8)
        # 1 x 1 x 2048
        x_for_capt = x = x.view(x.size(0), -1)
        # 2048
        x = self.fc(x)
        # 1000 (num_classes)
        return x_for_attn, x_for_capt, x

In [0]:
from torch.utils.model_zoo import load_url

inception_model = BeheadedInception3()

inception_url = 'https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth'
inception_model.load_state_dict(load_url(inception_url))

inception_model.eval()

Почему это вообще работает? Запустим модельку на картинке:

In [0]:
from matplotlib import pyplot as plt
from scipy.misc import imresize
%matplotlib inline
    
img = plt.imread('Flicker8k_Dataset/1000268201_693b08cb0e.jpg')
img = imresize(img, (299, 299)).astype('float32') / 255.
plt.imshow(img)

In [0]:
import requests
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'
labels = {int(key): value for (key, value) in requests.get(LABELS_URL).json().items()}

with torch.no_grad():
    img_tensor = torch.tensor(img.transpose([2, 0, 1]), dtype=torch.float32).unsqueeze(0)
    _, _, logits = inception_model(img_tensor)
    _, top_classes = logits.topk(5)

    print('; '.join(labels[ind.item()] for ind in top_classes.squeeze()))

Она выдает такие классы.

Подписи же к картинке такие:

In [0]:
with open('texts.txt') as f:
    text = f.readline().strip().split('\t')
print('\n'.join(text))

Загрузим данные:

In [0]:
source_field = Field(sequential=False, use_vocab=False, dtype=torch.float)
target_field = Field(init_token=BOS_TOKEN, eos_token=EOS_TOKEN)
path_field = Field(sequential=False, use_vocab=True)

fields = [('source', source_field), ('target', target_field), ('path', path_field)]

In [0]:
img_vectors = np.load('image_codes.npy')

examples = []
with open('texts.txt') as f_texts, open('sources.txt') as f_sources:
    for img, texts, source in zip(img_vectors, f_texts, f_sources):
        for text in texts.split('\t'):
            examples.append(Example.fromlist([img, target_field.preprocess(text), source.rstrip()], fields))

In [0]:
dataset = Dataset(examples, fields)

train_dataset, test_dataset = dataset.split(split_ratio=0.85)

print('Train size =', len(train_dataset))
print('Test size =', len(test_dataset))

target_field.build_vocab(train_dataset, min_freq=2)
path_field.build_vocab(dataset)
print('Target vocab size =', len(target_field.vocab))

train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), batch_sizes=(32, 512), shuffle=True, device=DEVICE, sort=False
)

**Задание** Реализуйте декодер для модели:

In [0]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, cnn_feature_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._cnn_to_h0 = nn.Linear(cnn_feature_size, rnn_hidden_dim)
        self._cnn_to_c0 = nn.Linear(cnn_feature_size, rnn_hidden_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=rnn_hidden_dim, num_layers=num_layers)
        self._out = nn.Linear(rnn_hidden_dim, vocab_size)

    def forward(self, encoder_output, inputs, hidden=None):
        ...
    
    def init_hidden(self, encoder_output):
        encoder_output = encoder_output.unsqueeze(0)
        return self._cnn_to_h0(encoder_output), self._cnn_to_c0(encoder_output)

Хак, чтобы все работало со старым циклом обучения:

In [0]:
def evaluate_model(model, iterator):
    return 0.

In [0]:
model = Decoder(vocab_size=len(target_field.vocab), cnn_feature_size=img_vectors.shape[1]).to(DEVICE)

pad_idx = target_field.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)

optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=30, val_iter=test_iter)

Проверим, что работает генерация:

In [0]:
batch = next(iter(test_iter))

img = path_field.vocab.itos[batch.path[0].item()]

img = plt.imread('Flicker8k_Dataset/' + img)
img = imresize(img, (299, 299)).astype('float32') / 255.
plt.imshow(img)

**Задание** Напишите цикл генерации из модели:

# Дополнительные материалы

## Статьи
Sequence to Sequence Learning with Neural Networks, Ilya Sutskever, et al, 2014 [[pdf]](https://arxiv.org/pdf/1409.3215.pdf)  
Show and Tell: A Neural Image Caption Generator, Oriol Vinyals et al, 2014 [[arxiv]](https://arxiv.org/abs/1411.4555)  
Scheduled Sampling for Sequence Prediction with Recurrent Neural Networks, Samy Bengio, et al, 2015 [[arxiv]](https://arxiv.org/abs/1506.03099)  
Neural Machine Translation of Rare Words with Subword Units, Rico Sennrich, 2015 [[arxiv]](https://arxiv.org/abs/1508.07909)  
Massive Exploration of Neural Machine Translation Architectures, Denny Britz, et al, 2017 [[pdf]](https://arxiv.org/pdf/1703.03906.pdf)

## Блоги
Neural Machine Translation (seq2seq) Tutorial [tensorflow/nmt](https://github.com/tensorflow/nmt)  
[A Word of Caution on Scheduled Sampling for Training RNNs](https://www.inference.vc/scheduled-sampling-for-rnns-scoring-rule-interpretation/)

## Видео
cs224n, [Machine Translation, Seq2Seq and Attention](https://www.youtube.com/watch?v=IxQtK2SjWWM)

## Разное
[The Annotated Encoder Decoder](https://bastings.github.io/annotated_encoder_decoder/)  
[Seq2Seq-Vis: A Visual Debugging Tool for Sequence-to-Sequence Models](http://seq2seq-vis.io)

# Сдача

[Форма для сдачи](https://goo.gl/forms/28RaQihilt5NChaI2)  
[Feedback](https://goo.gl/forms/9aizSzOUrx7EvGlG3)